<a href="https://colab.research.google.com/github/AndreyLFR/test/blob/main/T9_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание следующих букв

In [13]:
import re
import keras
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from keras.models import Sequential
from keras import callbacks

Получаем данные для обучения - текст

In [14]:
with open('/content/sample_data/train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я .]', '', text) # убираем все недопустимые символы
    text = text.replace('.', ' ')
    print(text)

Вы  лучший ответ на проблемы которые возникли в понедельник Думайте позитивно и верьте в свою способность достигать отличных результатов Если вы смогли в понедельник подняться с постели значит вы супер герой 


In [15]:
num_characters = 34 #33 буквы + пробел
inp_chars = 8 #количество символов, которые вводятся пользователем
outp_chars = 3 #количество символов, которые будем предсказывать

In [16]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)
tokenizer.fit_on_texts(text)
print(tokenizer.word_index)
data_chars = tokenizer.texts_to_matrix(text)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [17]:
X_train = np.array([data_chars[i:i+inp_chars, :] for i in range(data_chars.shape[0]-inp_chars)])
Y_train = data_chars[inp_chars:]

In [18]:
model_chars = Sequential()
model_chars.add(Input((inp_chars, num_characters)))
model_chars.add(SimpleRNN(500, activation='tanh'))
model_chars.add(Dense(num_characters, activation='softmax'))
#model_chars.summary()

model_chars.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

callb = callbacks.EarlyStopping(
                    monitor='accuracy',
                    mode='max', min_delta=0.001,
                    patience = 5)

history = model_chars.fit(X_train, Y_train, batch_size=32, epochs=50, callbacks=callb)

Epoch 1/50
7/7 [==============================] - 1s 20ms/step - loss: 3.3951 - accuracy: 0.0950
Epoch 2/50
7/7 [==============================] - 0s 20ms/step - loss: 2.3121 - accuracy: 0.3800
Epoch 3/50
7/7 [==============================] - 0s 19ms/step - loss: 1.7092 - accuracy: 0.5300
Epoch 4/50
7/7 [==============================] - 0s 19ms/step - loss: 1.2892 - accuracy: 0.6450
Epoch 5/50
7/7 [==============================] - 0s 22ms/step - loss: 1.0966 - accuracy: 0.7200
Epoch 6/50
7/7 [==============================] - 0s 22ms/step - loss: 0.8126 - accuracy: 0.8300
Epoch 7/50
7/7 [==============================] - 0s 20ms/step - loss: 0.6793 - accuracy: 0.8400
Epoch 8/50
7/7 [==============================] - 0s 20ms/step - loss: 0.5845 - accuracy: 0.8550
Epoch 9/50
7/7 [==============================] - 0s 19ms/step - loss: 0.4720 - accuracy: 0.9000
Epoch 10/50
7/7 [==============================] - 0s 19ms/step - loss: 0.4688 - accuracy: 0.8800
Epoch 11/50
7/7 [============

In [19]:
def buildPhrase(inp_str):
  for i in range(outp_chars):
    x_test = np.array([tokenizer.texts_to_matrix(inp_str[j]) for j in range(len(inp_str)-inp_chars, len(inp_str))])
    inp = x_test.reshape(1, inp_chars, num_characters)
    pred = model_chars.predict(inp)
    d = tokenizer.index_word[pred.argmax(axis=1)[0]]
    inp_str += d
  return inp_str

res = buildPhrase(inp_str="лучший ответ на")
print(res)

1/1 [==============================] - 0s 21ms/step
лучший ответ на пр


Была допущена ошибка в индексах, поэтому модель предсказывала неверный символ. Теперь все ОК:
смотрит на предыдущие 8 символов и предсказывает 3 следующие

# Предсказание слов


In [20]:
num_words = 100
inp_words = 3 #количество слов, которые вводятся пользователем
outp_words = 1 #количество слов, которые будем предсказывать

In [21]:
tokenizer_words = Tokenizer(num_words=num_words, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                              lower=True, split=' ', char_level=False)
tokenizer_words.fit_on_texts([text])
data_words = tokenizer_words.texts_to_sequences([text])
data_words = keras.utils.to_categorical(data_words[0], num_classes=num_words)
print(data_words)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
X_train_word = np.array([data_words[i:i+inp_words, :] for i in range(data_words.shape[0]-inp_words)])
Y_train_word = data_words[inp_words:]

In [23]:
model_words = Sequential()
model_words.add(Input((inp_words, num_words)))
model_words.add(SimpleRNN(128, activation='tanh'))
model_words.add(Dense(num_words, activation='softmax'))
#model_words.summary()

model_words.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model_words.fit(X_train_word, Y_train_word, batch_size=32, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 4.6193 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 4.5475 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 4.4758 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 4.4042 - accuracy: 0.1786
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 4.3323 - accuracy: 0.3929
Epoch 6/100
1/1 [==============================] - 0s 19ms/step - loss: 4.2599 - accuracy: 0.3929
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 4.1868 - accuracy: 0.5714
Epoch 8/100
1/1 [==============================] - 0s 299ms/step - loss: 4.1127 - accuracy: 0.6786
Epoch 9/100
1/1 [==============================] - 0s 12ms/step - loss: 4.0374 - accuracy: 0.7500
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 3.9605 - accuracy: 0.8214
Epoch 11

In [25]:
def buildPhrase(inp_str, tokenizer_words):
  res = inp_str
  data_words = tokenizer_words.texts_to_sequences([inp_str])
  data = keras.utils.to_categorical(data_words[0], num_classes=num_words)

  for i in range(outp_words):
    x_test = np.array([data[j] for j in range(len(data)-inp_words, len(data))])
    inp = x_test.reshape(1, inp_words, num_words)
    pred = model_words.predict(inp)
    indx = pred.argmax(axis=1)[0]
    res += ' ' + tokenizer_words.index_word[indx]
  return res

res = buildPhrase(inp_str="Вы  лучший ответ на", tokenizer_words=tokenizer_words)
print(res)

res = buildPhrase(inp_str="Думайте позитивно и верьте в", tokenizer_words=tokenizer_words)
print(res)

1/1 [==============================] - 0s 22ms/step
Вы  лучший ответ на проблемы
1/1 [==============================] - 0s 21ms/step
Думайте позитивно и верьте в свою


Все работает. На основании 3 предыдущих слов предсказывает 4 слово